In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score,train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler,OrdinalEncoder
from tensorflow import keras
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
import joblib

In [31]:
df=pd.read_csv('train\\[2023, 2024].csv')
df.columns

Index(['Driver', 'DriverNumber', 'LapTime', 'LapNumber', 'Stint', 'Compound',
       'TyreLife', 'Team', 'AirTemp', 'Humidity', 'Pressure', 'Rainfall',
       'TrackTemp', 'WindDirection', 'WindSpeed', 'Year', 'GrandPrix'],
      dtype='object')

In [18]:
drop_labels = ['Driver', 'DriverNumber', 'LapNumber', 'Stint', 'Compound',
                'TyreLife', 'Team', 'AirTemp', 'Humidity', 'Pressure', 'Rainfall',
                'TrackTemp', 'WindDirection', 'WindSpeed', 'Year', 'GrandPrix']

In [32]:
for cols in df.columns:
    print(cols+'    \t\t\t'+str(df[cols].isna().sum()))

Driver    			0
DriverNumber    			0
LapTime    			434
LapNumber    			0
Stint    			0
Compound    			0
TyreLife    			0
Team    			0
AirTemp    			0
Humidity    			0
Pressure    			0
Rainfall    			0
TrackTemp    			0
WindDirection    			0
WindSpeed    			0
Year    			0
GrandPrix    			0


In [33]:
df.dropna(axis=0,how='any',inplace=True)

In [34]:
for cols in df.columns:
    print(cols+'    \t\t'+str(df[cols].isna().sum()))

Driver    		0
DriverNumber    		0
LapTime    		0
LapNumber    		0
Stint    		0
Compound    		0
TyreLife    		0
Team    		0
AirTemp    		0
Humidity    		0
Pressure    		0
Rainfall    		0
TrackTemp    		0
WindDirection    		0
WindSpeed    		0
Year    		0
GrandPrix    		0


In [35]:
#Making sure the features are on the same scale and encoding some features

scale_cols = ['LapNumber', 'TyreLife', 'AirTemp', 'TrackTemp', 'WindSpeed']
categorical_cols = ['Driver','GrandPrix', 'Compound', 'DriverNumber', 'Team', 'Stint', 'Rainfall']

encoder = OrdinalEncoder()
scaler = StandardScaler()

for cols in categorical_cols:
    df[cols]=encoder.fit_transform(np.array(df[cols]).reshape(-1,1))

df[scale_cols]=scaler.fit_transform(df[scale_cols])


In [36]:
df.shape

(31828, 17)

In [37]:
for cols in df.columns:
    print(cols+'    \t\t'+str(df[cols].isna().sum()))

Driver    		0
DriverNumber    		0
LapTime    		0
LapNumber    		0
Stint    		0
Compound    		0
TyreLife    		0
Team    		0
AirTemp    		0
Humidity    		0
Pressure    		0
Rainfall    		0
TrackTemp    		0
WindDirection    		0
WindSpeed    		0
Year    		0
GrandPrix    		0


In [38]:
x=df.drop(labels=['LapTime'],axis=1).dropna(axis=0,how='any')
y=df.drop(labels=drop_labels,axis=1).dropna(axis=0,how='any')
x=x.to_numpy()
y=y.to_numpy()

print(x.shape)
print(y.shape)

(31828, 16)
(31828, 1)


In [39]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

In [40]:
print(x_train.shape)
print(y_train.shape)

(25462, 16)
(25462, 1)


**Linear Regression**

In [41]:
rgs = LinearRegression(positive=True,fit_intercept=True)
rgs.fit(x_train,y_train)
y_pred = rgs.predict(x_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')
score = cross_val_score(estimator=rgs,X=x_train,y=y_train,cv=10)
print('\nCross Validation: ',score)
print('\nMean accuracy: ',np.mean(np.array(score*100)))

Mean Squared Error: 2901.235206099388
R-squared: 0.0041859181610715

Cross Validation:  [-0.01438761  0.00465262 -0.02283331  0.00374731 -0.0116209   0.00419242
  0.00386632  0.0036159   0.00226658  0.00399969]

Mean accuracy:  -0.22500980087577616


**Random Forest**

In [42]:
rndm = RandomForestRegressor(n_estimators=10,random_state=42)
rndm.fit(x_train,y_train)
y_pred=rndm.predict(x_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')

score = cross_val_score(estimator=rndm,X=x_train,y=y_train)
print('\nCross Validation: ',score)
print('\nMean accuracy: ',np.mean(np.array(score)))

c:\Users\Ghost\anaconda3\envs\py310\lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Mean Squared Error: 27.95444713460416
R-squared: 0.9904049723207078


c:\Users\Ghost\anaconda3\envs\py310\lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\Ghost\anaconda3\envs\py310\lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\Ghost\anaconda3\envs\py310\lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\Ghost\anaconda3\envs\py310\lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Pleas


Cross Validation:  [0.99343207 0.99157466 0.98548616 0.9976016  0.99699725]

Mean accuracy:  0.9930183462428872


**MLP Regression**

In [43]:
ann =MLPRegressor(hidden_layer_sizes=(256,64),activation='relu',solver='adam',learning_rate='invscaling',
                  learning_rate_init=0.003)

ann.fit(x_train,y_train)
y_pred=ann.predict(x_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')

score = cross_val_score(estimator=ann,X=x_train,y=y_train)
print('\nCross Validation: ',score)
print('\nMean accuracy: ',np.mean(np.array(score)))

c:\Users\Ghost\anaconda3\envs\py310\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Mean Squared Error: 4728.805670483614
R-squared: 0.012043199109971314


c:\Users\Ghost\anaconda3\envs\py310\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Ghost\anaconda3\envs\py310\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Ghost\anaconda3\envs\py310\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Ghost\anaconda3\envs\py310\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:162


Cross Validation:  [ 0.00357945 -0.01664747  0.00980712 -0.09270481  0.01372772]

Mean accuracy:  -0.01644759868866219


In [43]:
joblib.dump(rndm,'laptime_predictor.sav')

['laptime_predictor.sav']

In [44]:
predictor = joblib.load('laptime_predictor.sav')

In [45]:
df2=pd.read_csv('test\\2024 Singapore Grand Prix.csv',index_col=None)
df2.columns

Index(['Driver', 'DriverNumber', 'LapTime', 'LapNumber', 'Stint', 'Compound',
       'TyreLife', 'Team', 'AirTemp', 'Humidity', 'Pressure', 'Rainfall',
       'TrackTemp', 'WindDirection', 'WindSpeed', 'Year', 'GrandPrix'],
      dtype='object')

In [116]:
x1=df2.iloc[0:5].drop(labels=['LapTime'],axis=1)

#Making sure the features are on the same scale and encoding some features

scale_cols = ['LapNumber', 'TyreLife']
categorical_cols = ['Driver','GrandPrix', 'Compound', 'DriverNumber', 'Team', 'Stint']

encoder = OrdinalEncoder()
scaler = StandardScaler()

for cols in categorical_cols:
    x1[cols]=encoder.fit_transform(np.array(x1[cols]).reshape(-1,1))

x1[scale_cols]=scaler.fit_transform(x1[scale_cols])

x2 = df2['LapTime'].iloc[0:5].values
y1=predictor.predict(x1)
print(y1)
print(df2['LapTime'].iloc[0:5])
diff = y1-df2['LapTime'].iloc[0:5]
print(diff)
error = (diff/df2['LapTime'].iloc[0:5])*100
print(f'Perecent Error: \n{error}')


[120.2455  96.9585  97.1129  98.4245  97.0764]
0    96.928
1    95.953
2    96.567
3    94.925
4    96.060
Name: LapTime, dtype: float64
0    23.3175
1     1.0055
2     0.5459
3     3.4995
4     1.0164
Name: LapTime, dtype: float64
Perecent Error: 
0    24.056516
1     1.047909
2     0.565307
3     3.686595
4     1.058089
Name: LapTime, dtype: float64


c:\Users\Ghost\anaconda3\envs\py310\lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


In [128]:
df_pred = pd.DataFrame(data=(x2,y1,error),)
df_pred

,0,1,2,3,4
0,96.928000,95.953000,96.567000,94.925000,96.060000
1,120.245500,96.958500,97.112900,98.424500,97.076400
2,24.056516,1.047909,0.565307,3.686595,1.058089
